In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('iris.csv')

In [3]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [28]:
df.columns = ['sepal_length','sepal_width','petal_length','petal_width','target']
df.columns

Index([u'sepal_length', u'sepal_width', u'petal_length', u'petal_width',
       u'target'],
      dtype='object')

In [29]:
X = df.drop('target', axis=1)
y = df['target'].apply(int)

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [31]:
import tensorflow as tf

In [32]:
feat_cols = []
for col in X.columns:
    feat_cols.append(tf.feature_column.numeric_column(col))

In [33]:
feat_cols

[_NumericColumn(key='sepal_length', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='sepal_width', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='petal_length', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='petal_width', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [43]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=30,num_epochs=5,shuffle=True)
# classifier = tf.estimator.DNNClassifier(hidden_units=[10, 20, 10], n_classes=3,feature_columns=feat_cols)
classifier = tf.estimator.DNNClassifier(hidden_units=[30,30,30], n_classes=3, feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x12ac86dd0>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_device_fn': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/var/folders/84/hkd04yfn69l8tdnk9djgbxl80000gn/T/tmpGO97pt', '_train_distribute': None, '_save_summary_steps': 100}


In [44]:
classifier.train(input_fn=input_func,steps=50)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/84/hkd04yfn69l8tdnk9djgbxl80000gn/T/tmpGO97pt/model.ckpt.
INFO:tensorflow:loss = 42.302998, step = 1
INFO:tensorflow:Saving checkpoints for 18 into /var/folders/84/hkd04yfn69l8tdnk9djgbxl80000gn/T/tmpGO97pt/model.ckpt.
INFO:tensorflow:Loss for final step: 6.108695.


In [45]:
# pred_fn = tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=len(X_test),shuffle=False)
pred_fn = tf.estimator.inputs.pandas_input_fn(x=X_test, batch_size=len(X_test), shuffle=False)

In [46]:
note_predictions = list(classifier.predict(input_fn=pred_fn))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/84/hkd04yfn69l8tdnk9djgbxl80000gn/T/tmpGO97pt/model.ckpt-18
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [47]:
note_predictions[0]

{'class_ids': array([1]),
 'classes': array(['1'], dtype=object),
 'logits': array([-1.8368583 ,  0.78761566,  0.6946594 ], dtype=float32),
 'probabilities': array([0.0365365 , 0.5041056 , 0.45935786], dtype=float32)}

In [48]:
final_preds  = []
for pred in note_predictions:
    final_preds.append(pred['class_ids'][0])

In [49]:
final_preds

[1,
 0,
 1,
 0,
 1,
 0,
 2,
 1,
 2,
 1,
 2,
 1,
 2,
 0,
 2,
 2,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 2,
 0,
 1,
 0,
 0,
 2,
 1,
 0,
 1,
 0,
 2,
 1,
 2,
 1]

In [50]:
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,final_preds))
print(classification_report(y_test,final_preds))

[[17  0  0]
 [ 0 17  0]
 [ 0  0 11]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        17
           1       1.00      1.00      1.00        17
           2       1.00      1.00      1.00        11

   micro avg       1.00      1.00      1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45

